# Problem

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv("I:\puzzle1\product.csv.gz", compression='gzip')
product_data = raw_data[raw_data.type == 'PARENT']
product_data.head(5)

,productName,type,productSku,description,parentId,productCategoryIds,thumbnail,fullsize,label,cogs,sellingPrice,size,style,productGroupId
7,125TH Boot Double Cap Toe,PARENT,12152-202,This double cap boot features the same iconic ...,NaN,NaN,https://www.florsheim.com/shop/thumbnails/1215...,https://www.florsheim.com/shop/thumbnails/1215...,NaN,NaN,275.0,NaN,NaN,1
17,125TH Oxford Cap Toe,PARENT,12146-975,The 125th Cap Toe Oxford features a stunning e...,NaN,NaN,https://www.florsheim.com/shop/thumbnails/1214...,https://www.florsheim.com/shop/thumbnails/1214...,NaN,NaN,250.0,NaN,NaN,2
30,ACE,PARENT,1072A,ACE is produced from 100% US-grown cotton. Aft...,NaN,NaN,https://zealoptics.scene7.com/is/image/ZealOpt...,https://zealoptics.scene7.com/is/image/ZealOpt...,NaN,NaN,169.0,NaN,NaN,3
42,Adidas Crazylight Boost 2018 Shoes,PARENT,10041,LIGHTWEIGHT LOW TOPS WITH ENERGY RETURN.\r\nDe...,NaN,['100005'],https://www.adidas.com/dis/dw/image/v2/aaqx_pr...,https://www.adidas.com/dis/dw/image/v2/aaqx_pr...,NaN,NaN,100.0,NaN,NaN,4
61,Adidas Crazylight Boost 2018 Shoes,PARENT,10041,LIGHTWEIGHT LOW TOPS WITH ENERGY RETURN.\r\nDe...,NaN,['100005'],https://www.adidas.com/dis/dw/image/v2/aaqx_pr...,https://www.adidas.com/dis/dw/image/v2/aaqx_pr...,NaN,NaN,100.0,NaN,NaN,5


In [3]:
category_data = pd.read_csv("I:\puzzle1\category.csv.gz", compression='gzip')
category_data.head(5)

,categoryName,categoryId,categoryDescription,categoryMedia
0,accessories,1002,accessories,https://s3.amazonaws.com/caas-demo-products/b2...
1,activewear,1008,activewear,https://s3.amazonaws.com/caas-demo-products/b2...
2,apparel,1003,apparel,https://s3.amazonaws.com/caas-demo-products/b2...
3,Apparel,100004,Apparel,https://s3.amazonaws.com/caas-demo-products/b2...
4,backpacks,1018,backpacks,https://s3.amazonaws.com/caas-demo-products/b2...


In [4]:
raw_data = pd.read_csv("I:\puzzle1\product_distance_prod.csv")
distance_data = raw_data[raw_data.tenant_id == 'caas20demo']
distance_data.head(5)

,id,sku_1,sku_2,distance,tenant_id
0,1,46601,46606,0.403930,caas20demo
1,2,46601,46605,1.218187,caas20demo
2,3,46601,10415,1.330609,caas20demo
3,4,46601,205402,1.416728,caas20demo
4,5,46601,10414,1.335640,caas20demo


### Category distance

$ distance[category(i), category(j)] = \frac {\sum_{p \in category(i), q \in category(j)} {distance(p, q)} } { || p, q||} $ 

For each category, find most similar top 4 categories with their name?

# Answer

#### 1 read product

In [5]:
product_data = pd.read_csv("I:\puzzle1\product.csv.gz", compression='gzip')
# product_data = raw_data[raw_data.type == 'PARENT']
product_data[product_data.productCategoryIds.isnull()]
product_data = product_data[product_data.productCategoryIds.notnull()]

product_data.index = range(len(product_data))
for indexs in range(len(product_data)):
    product_data.loc[indexs, ('productCategoryIds')] = (product_data['productCategoryIds'][indexs].split("['")[1]).split("']")[0]
    product_data.loc[indexs, ('productCategoryIds')] = product_data['productCategoryIds'][indexs].split("',")[0]
product_data = product_data[['productSku', 'productCategoryIds']]
product_data.head(5)

,productSku,productCategoryIds
0,78850,100006
1,4827020,1004
2,45616,100003
3,2833888,1005
4,78829,100005


####  2 Combine any two of the types.

In [6]:
category_set = set(product_data.productCategoryIds)
category_list = list(category_set)
category_list_two = list()
for i in range(len(category_list)):
    for j in range(len(category_list)):
        if i < j:
            category_list_two.append([category_list[i], category_list[j]])

category_df_two = pd.DataFrame(category_list_two, columns=['category_1', 'category_2'])
category_df_two.index = range(len(category_df_two))
category_df_two['distance'] = 0.0
category_df_two.head(5)

,category_1,category_2,distance
0,9f1a9306-be90-4149-9307-2a63b3a1d89d,1039,0.0
1,9f1a9306-be90-4149-9307-2a63b3a1d89d,1011,0.0
2,9f1a9306-be90-4149-9307-2a63b3a1d89d,100012,0.0
3,9f1a9306-be90-4149-9307-2a63b3a1d89d,100016,0.0
4,9f1a9306-be90-4149-9307-2a63b3a1d89d,1038,0.0


#### 3 read product_distance

In [7]:
distance_data = pd.read_csv("I:\puzzle1\product_distance_prod.csv")
distance_data = distance_data[((distance_data['sku_1'].isin(product_data.productSku)) & (
    distance_data['sku_2'].isin(product_data.productSku)))]
distance_data.index = range(len(distance_data))
distance_data.head(5)

,id,sku_1,sku_2,distance,tenant_id
0,1,46601,46606,0.403930,caas20demo
1,2,46601,46605,1.218187,caas20demo
2,8,46601,4623747,1.375010,caas20demo
3,9,46601,4601710,1.401897,caas20demo
4,10,46601,3626596,2.426523,caas20demo


#### 4 Calculating distance

In [8]:
for indexs in range(len(category_df_two)):

    product_sku_1 = product_data[product_data['productCategoryIds'] == category_df_two['category_1'][indexs]]
    product_sku_2 = product_data[product_data['productCategoryIds'] == category_df_two['category_2'][indexs]]
    product_set_sku = set(product_sku_1.productSku) | set(product_sku_2.productSku)

    count = 0
    distance = 0.0
    for i in range(len(distance_data)):
        if distance_data['sku_1'][i] in product_set_sku and distance_data['sku_2'][i] in product_set_sku:
            count += 1
            distance = distance_data['distance'][i]+distance
    if(count == 0):
        category_df_two.loc[indexs, ('distance')] = 0
    else:
        category_df_two.loc[indexs, ('distance')] = distance/count
#     print indexs
#category_df_two.to_csv('category_distance.csv', index=False)
category_df_two.head(5)

,category_1,category_2,distance
0,9f1a9306-be90-4149-9307-2a63b3a1d89d,1039,1.035946
1,9f1a9306-be90-4149-9307-2a63b3a1d89d,1011,1.087426
2,9f1a9306-be90-4149-9307-2a63b3a1d89d,100012,1.220779
3,9f1a9306-be90-4149-9307-2a63b3a1d89d,100016,0.899766
4,9f1a9306-be90-4149-9307-2a63b3a1d89d,1038,1.145529


#### 5 dic of categoryId_to_categoryName

In [9]:
category = pd.read_csv("I:\puzzle1\category.csv.gz", compression='gzip')

id_to_catename = {}
for index in range(len(category)):
    key = category['categoryId'].iloc[index]
    values = str(category['categoryName'].iloc[index])
    id_to_catename[key] = values

####  6 A dictionary is used to store the maximum 4 values

In [29]:
sort_distance = {}
for index in range(len(category_df_two)):

    category = str(category_df_two['category_1'].iloc[index])

    if(category in set(category_df_two['category_1'])) and (category in set(category_df_two['category_2'])):
        data1 = category_df_two[category_df_two['category_1'] == category]
        data2 = category_df_two[category_df_two['category_2'] == category]
#         print 'one'
        data = pd.concat([data1, data2])

    elif(category in set(category_df_two['category_1'])) :
        data = category_df_two[category_df_two['category_1'] == category]
#         print 'two'

    elif(category in set(category_df_two['category_2'])):
        data = category_df_two[category_df_two['category_2'] == category]
#         print 'three'

    data = data.sort_values(by='distance', ascending=False)
    if (len(data > 4)):
        data = data[:4]

    if category in id_to_catename.keys():
        categoryname = id_to_catename[category]

    similar = pd.concat([data['category_1'], data['category_2']])
    similar = set(similar)
    similar.remove(category)
    similar_list = list(similar)


    for i in range(len(similar_list)):
        if similar_list[i] in id_to_catename.keys():
            similar_list[i] = id_to_catename[similar_list[i]]

    sort_distance[categoryname] = similar_list

####  7 The result we need is sort_df_distance.

In [31]:
sort_list_distance = list()

for key,values in sort_distance.items():
#     print key
#     values = sort_distance[key]
#     print values  
    values.insert(0,key)
    sort_list_distance.append(values)
#     print values
sort_df_distance = pd.DataFrame(sort_list_distance)
sort_df_distance.columns = [['category', 'simliary_1', 'simliary_2','simliary_3','simliary_4']]
# sort_df_distance.to_csv('sort_distance.csv',index=False)
sort_df_distance.head(10)

,category,simliary_1,simliary_2,simliary_3,simliary_4
0,Basketball,Camping / Hiking,Hunting,accessories,tools
1,Cycling,Hunting,accessories,tools,Camping / Hiking
2,Hunting,watches,apparel,makeup,necklaces
3,jeans,Hunting,watches,accessories,tools
4,Select,Fitness,Hunting,Outdoor,Camping / Hiking
5,watches,apparel,makeup,Hunting,tools
6,skincare,watches,Hunting,accessories,tools
7,footwear,Camping / Hiking,Hunting,accessories,tools
8,heels,Camping / Hiking,Hunting,accessories,tools
9,Electronics,accessories,Hunting,tools,Camping / Hiking
